# User Satsfaction Score Prediction

In [5]:
import pandas as pd
import os
import sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [6]:
# Load the xdr_cleaned dataset
data = pd.read_csv('../data/xdr_cleaned.csv')

In [7]:
# Import modules
from user_engagement_analysis import UserEngagementAnalysis
from experience_analytics import ExperienceAnalytics
from satisfaction_analytics import UserSatisfactionAnalytics

Connection to database successful!


In [9]:
# Create instances of engagement and user experience classes
engagement_analysis = UserEngagementAnalysis(data)
experience_analysis = ExperienceAnalytics(data)
satisfaction_analysis = UserSatisfactionAnalytics()

In [10]:
# Normalize and cluster
# Aggregate engagement metrics
engagement_analysis.aggregate_metrics()
engagement_analysis.metrics.head()
engagement_analysis.normalize_and_cluster(n_clusters=3)

In [13]:
# Computer Engagement Score
engagement_features = [
    'total_session_duration',
    'total_download_traffic',
    'total_upload_traffic', 
    'sessions_frequency']

# Identify least engaged cluster
least_engaged_cluster = engagement_analysis.cluster_centers[engagement_features].sum(axis=1).idxmin()

# Compute engagement scores
engagement_scores = satisfaction_analysis.compute_score(
    engagement_analysis.metrics, 
    engagement_analysis.cluster_centers, 
    engagement_features, 'Engagement_Score',
    least_engaged_cluster)

In [14]:
#Check for duplicates in the 'MSISDN/Number' column and count them
duplicate_count = engagement_analysis.metrics.duplicated('MSISDN/Number').sum()
print(f"Number of duplicate entries in 'MSISDN/Number': {duplicate_count}")

Number of duplicate entries in 'MSISDN/Number': 0


In [15]:
engagement_scores.head(20)

,MSISDN/Number,Engagement_Score
0,3.361370e+10,3.376757e+12
1,3.361370e+10,1.851072e+08
2,3.361370e+10,9.416116e+08
3,3.361370e+10,4.739266e+08
4,3.361370e+10,6.141363e+07
5,3.361370e+10,5.971572e+08
6,3.361371e+10,1.546607e+08
7,3.361371e+10,2.179564e+07
8,3.361371e+10,3.442113e+08
9,3.361371e+10,1.355662e+08


In [16]:
# User experience clusters
# Define features
experience_features = ['TCP Retransmission', 'RTT', 'Throughput']

# aggregate and cluster
experience_df, cluster_centers = experience_analysis.k_means_clustering(experience_features)
# rename cluster column
experience_df.rename(columns={'Cluster':'cluster'}, inplace=True)
cluster_centers.rename(columns={'Cluster':'cluster'}, inplace=True)
# Identify worst experience cluster
worst_cluster = cluster_centers[experience_features].sum(axis=1).idxmin()

In [ ]:
# Computer User experience scores
experience_scores = satisfaction_analysis.compute_score(
    experience_df, 
    cluster_centers, 
    experience_features, 'Experience_Score',
    worst_cluster)